# Chapter 14. 분산형 공유 변수

- 스파크의 저수준 API에는 RDD 인터페이스 외에도 두 번째 유형인 '분산형 공유 변수'가 있음
    - 브로드캐스트 변수 : 모든 워커 노드에 큰 값을 저장하므로 재전송 없이 많은 스파크 액션에서 재사용할 수 있음
    - 어큐뮬레이터 : 모든 태스크의 데이터를 공유 결과에 추가할 수 있음

# 14.1 브로드캐스트 변수
- 변하지 않는 값(불변성 값)을 클로저 함수의 변수로 캡슐화하지 않고 클러스터에서 효율적으로 공유하는 방법 제공
- 모든 태스크마다 직렬화하지 않고 클러스터의 모든 머신에 캐시하는 불변성 공유 변수

In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.2:4040
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1686177429554)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3aaf14d6


In [2]:
val myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
val words = spark.sparkContext.parallelize(myCollection, 2)

myCollection: Array[String] = Array(Spark, The, Definitive, Guide, :, Big, Data, Processing, Made, Simple)
words: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[0] at parallelize at <console>:24


In [3]:
val supplementalData = Map("Spark" -> 1000, "Definitive" -> 200, "Big" -> -300, "Simple" -> 100)

supplementalData: scala.collection.immutable.Map[String,Int] = Map(Spark -> 1000, Definitive -> 200, Big -> -300, Simple -> 100)


In [4]:
val suppBroadcast = spark.sparkContext.broadcast(supplementalData)

suppBroadcast: org.apache.spark.broadcast.Broadcast[scala.collection.immutable.Map[String,Int]] = Broadcast(0)


In [5]:
suppBroadcast.value

res1: scala.collection.immutable.Map[String,Int] = Map(Spark -> 1000, Definitive -> 200, Big -> -300, Simple -> 100)


In [6]:
words.map(word => (word, suppBroadcast.value.getOrElse(word, 0))).sortBy(wordPair => wordPair._2).collect()

java.lang.InternalError:  java.lang.IllegalAccessException: final field has no write access: $Lambda$2604/0x0000000801aed050.arg$1/putField, from class java.lang.Object (module java.base)

# 14.2 어큐뮬레이터
- 트랜스포메이션 내부의 다양한 값을 갱신하는 데 사용
- 스파크 클러스터에서 로우 단위로 안전하게 값을 갱신할 수 있는 변경 가능한 변수를 제공
- 어큐뮬레이터의 값은 액션을 처리하는 과정에서만 갱신되며 각 태스크엣 한 번만 갱신하도록 제어됨

## 14.2.1 기본 예제

In [9]:
case class Flight(DEST_COUNTRY_NAME: String, ORIGIN_COUNTRY_NAME: String, count: BigInt)
val flights = spark.read.parquet("./flight-data/parquet/2010-summary.parquet").as[Flight]

defined class Flight
flights: org.apache.spark.sql.Dataset[Flight] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [10]:
import org.apache.spark.util.LongAccumulator

val accUnnamed = new LongAccumulator
val acc = spark.sparkContext.register(accUnnamed)

import org.apache.spark.util.LongAccumulator
accUnnamed: org.apache.spark.util.LongAccumulator = LongAccumulator(id: 26, name: None, value: 0)
acc: Unit = ()


In [11]:
val accChina = new LongAccumulator
val accChina2 = spark.sparkContext.longAccumulator("China")

spark.sparkContext.register(accChina, "China")

accChina: org.apache.spark.util.LongAccumulator = LongAccumulator(id: 28, name: Some(China), value: 0)
accChina2: org.apache.spark.util.LongAccumulator = LongAccumulator(id: 27, name: Some(China), value: 0)


In [12]:
def accChinaFunc(flight_row: Flight) = {
    val destination = flight_row.DEST_COUNTRY_NAME
    val origin = flight_row.ORIGIN_COUNTRY_NAME
    
    if(destination == "China") {
        accChina.add(flight_row.count.toLong)
    }
    if(origin == "China") {
        accChina.add(flight_row.count.toLong)
    }
}

accChinaFunc: (flight_row: Flight)Unit


In [13]:
flights.foreach(flight_row => accChinaFunc(flight_row))

java.lang.InternalError:  java.lang.IllegalAccessException: final field has no write access: $Lambda$3763/0x0000000801171400.arg$1/putField, from class java.lang.Object (module java.base)

In [14]:
accChina.value

res5: Long = 0


## 14.4.2 사용자 정의 어큐뮬레이터
- 직접 정의하려면 AccumulatorV2 클래스를 상속받아야 함

In [16]:
//짝수값만 더하는 예제

import scala.collection.mutable.ArrayBuffer
import org.apache.spark.util.AccumulatorV2

val arr = ArrayBuffer[BigInt]()

class EvenAccumulator extends AccumulatorV2[BigInt, BigInt] {
    private var num:BigInt = 0
    def reset(): Unit = {
        this.num=0
    }
    def add(intValue: BigInt): Unit = {
        if(intValue % 2 == 0) {
            this.num+=intValue
        }
    }
    def merge(other: AccumulatorV2[BigInt, BigInt]): Unit = {
        this.num+=other.value
    }
    def value():BigInt = {
        this.num
    }
    def copy(): AccumulatorV2[BigInt,BigInt] = {
        new EvenAccumulator
    }
    def isZero():Boolean = {
        this.num == 0
    }
}

val acc = new EvenAccumulator
val newAcc = sc.register(acc, "evenAcc")

import scala.collection.mutable.ArrayBuffer
import org.apache.spark.util.AccumulatorV2
arr: scala.collection.mutable.ArrayBuffer[BigInt] = ArrayBuffer()
defined class EvenAccumulator
acc: EvenAccumulator = EvenAccumulator(id: 37, name: Some(evenAcc), value: 0)
newAcc: Unit = ()


In [17]:
acc.value

res6: BigInt = 0


In [18]:
flights.foreach(flight_row => acc.add(flight_row.count))
acc.value

java.lang.InternalError:  java.lang.IllegalAccessException: final field has no write access: $Lambda$4208/0x00000008020f8208.arg$1/putField, from class java.lang.Object (module java.base)